In [1]:
# Uninstall bqplot, dash TODO

import sys
sys.dont_write_bytecode = True

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import ipyvuetify as v
import traitlets

import ipyvue

import ipywidgets as widgets
from ipywidgets import jslink
import numpy as np
from bqplot import pyplot as plt
import bqplot

# Following two lines make sure anything imported from .py scripts 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)
%reload_ext autoreload

# %load_ext autoreload
# %autoreload 2
from board_viz import ReplayGame, InteractiveGame
# from interactive_board_viz import PlayInteractiveGame
from isolation import Board
from test_players import RandomPlayer, HumanPlayer, Player
from custom_player import CustomPlayer
from evaluation_functions import (
    OpenMoveEvalFn,
    DefensiveEvalFn,
    OffensiveEvalFn,
    DefenseToOffenseEvalFn,
    OffenseToDefenseEvalFn,
)
import player_submission_tests as tests


# Board visualization with ipywidgets
import copy
import time
from time import sleep
import functools

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Label, Button, GridspecLayout
from ipywidgets import Button, GridBox, Layout, ButtonStyle
from IPython.display import display, clear_output, HTML

import platform
# import io
from io import StringIO

In [2]:
def get_details(name):
    if name in {'11','12','13'}:
        color = 'SpringGreen'
    elif name in {'21','22','23'}:
        color = 'tomato'
    elif name == 'q1':
        color = '#bdffbd'
        name = ' '
    elif name == 'q2':
        color = '#ffb6ae'
        name = ' '
    elif name == 'X':
        color = 'black'
    elif name == 'O':
        color = 'orange'
        name = ' '
    else:
        color = 'Lavender'
    style = ButtonStyle(button_color=color)
    return name, style

def create_cell(button_name='', grid_loc=None, click_callback=None):
    layout = Layout(width='auto', height='auto')
    name, style = get_details(button_name)
    button = Button(description=name,layout = layout, style=style)
    button.x, button.y = grid_loc
    if click_callback: button.on_click(click_callback)
    return button

def get_viz_board_state(game, show_legal_moves):
    board_state = game.get_state()
    legal_moves = game.get_active_moves()
    active_player = 'q1' if game.__active_player__ is game.__player_1__ else 'q2'
    if show_legal_moves:
        for moves in legal_moves:
            for r,c in moves:
                board_state[r][c] = active_player
    return board_state

def create_board_gridbox(game, show_legal_moves, click_callback=None):
    h, w = game.height, game.width
    board_state = get_viz_board_state(game, show_legal_moves)

    grid_layout = GridspecLayout(n_rows=h,
                                 n_columns=w,
                                 grid_gap='2px 2px',
                                 width='480px',
                                 height='480px',
                                 justify_content='center'
                                )
    for r in range(h):
        for c in range(w):
            cell = create_cell(board_state[r][c], grid_loc=(r,c), click_callback=click_callback)
            grid_layout[r,c] = cell

    return grid_layout

In [6]:
# Board visualization with ipywidgets
import copy
from time import sleep
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Label, Button, GridspecLayout
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Output
from IPython.display import display, clear_output
from numpy import isin

from isolation import Board
from test_players import Player, RandomPlayer, HumanPlayer
from custom_player import CustomPlayer

import time
import platform

# import io
from io import StringIO

# import resource
if platform.system() != "Windows":
    import resource


def get_details(name):
    if name in {"11", "12", "13"}:
        color = "SpringGreen"
    elif name in {"21", "22", "23"}:
        color = "tomato"
    elif name == "q1":
        color = "#bdffbd"
        name = " "
    elif name == "q2":
        color = "#ffb6ae"
        name = " "
    elif name == "X":
        color = "black"
    elif name == "O":
        color = "orange"
        name = " "
    else:
        color = "Lavender"
    style = ButtonStyle(button_color=color)
    return name, style


def create_cell(button_name="", grid_loc=None, click_callback=None):
    layout = Layout(width="auto", height="auto")
    name, style = get_details(button_name)
    button = Button(description=name, layout=layout, style=style)
    button.x, button.y = grid_loc
    if click_callback:
        button.on_click(click_callback)
    return button


def get_viz_board_state(game, show_legal_moves):
    board_state = game.get_state()
    legal_moves = game.get_active_moves()
    active_player = "q1" if game.__active_player__ is game.__player_1__ else "q2"
    if show_legal_moves:
        for moves in legal_moves:
            for r, c in moves:
                board_state[r][c] = active_player
    return board_state


def create_board_gridbox(game, show_legal_moves, click_callback=None):
    h, w = game.height, game.width
    board_state = get_viz_board_state(game, show_legal_moves)

    grid_layout = GridspecLayout(
        n_rows=h,
        n_columns=w,
        grid_gap="2px 2px",
        width="480px",
        height="480px",
        justify_content="center",
    )
    for r in range(h):
        for c in range(w):
            cell = create_cell(board_state[r][c], grid_loc=(r, c), click_callback=click_callback)
            grid_layout[r, c] = cell

    return grid_layout

def null_callback(b):
    """ Initial callback function before player options are set
    """
    # global output section widget
    with out:
        output = "Please Select Players first \n"
        out.append_stdout(output)


class PlayInteractiveGame:
    """This class is used to play the game interactively (only works in jupyter)"""

    def __init__(
        self, 
        player1 = Player("Player1"), 
        opponent = Player("Player2"), 
        show_legal_moves = False, 
        output_section = widgets.Output(
            layout={
                'border': '1px solid black', 
                'overflow_y': 'auto',
                'height': '80px',
                'width': '480px'
            }
        )
    ):
        self.player1 = player1
        self.opponent = opponent
        self.game = Board(player1, opponent)
        self.output_section = output_section
        self.width = self.game.width
        self.height = self.game.height
        self.show_legal_moves = show_legal_moves
        self.__click_count = 0
        self.__move = []
        self.callback_func = self.select_callback_func(player1, opponent)
        self.gridb = create_board_gridbox(
            self.game, self.show_legal_moves, click_callback=self.callback_func
        )
        self.visualized_state = None
        self.game_is_over = False


    def null_callback(self, b):
        """ Initial callback function before player options are set
        """
        # global output section widget
        with out:
            output = "Please Select Players first \n"
            out.append_stdout(output)

    def select_callback_func(self, player1, player2):
        """ Function to determine which callback function to use based on the type of players
            involved in the game
        """

        if isinstance(player1, Player) and isinstance(player2, RandomPlayer):
#             print(f"Human vs Random Player")
            return self.select_move

        elif isinstance(player1, RandomPlayer) and isinstance(player2, CustomPlayer):
            raise NotImplementedError("Random vs Random not implemented")

        elif isinstance(player1, RandomPlayer) and isinstance(player2, RandomPlayer):
            raise NotImplementedError("Random vs Random not implemented")

        elif isinstance(player1, RandomPlayer) and isinstance(player2, Player):
            raise NotImplementedError("Random (P1) vs Human (P2) not implemented")

        elif isinstance(player1, Player) and isinstance(player2, Player):
            # Human Vs. Human
            return self.select_move
        
        elif not player1 and not player2:
            return self.null_callback
        
        else:
            return None
        

    def __reset_turn(self):
        self.__click_count = 0
        self.__move = []
        with self.output_section:
            self.output_section.append_stdout("-" * 50 + "\n")
#         self.output_section.clear_output()

    def run_cpu_ai_game(self, run_game = False):
        """ Function to run a game against the Random CPU and Custom AI Agent
        """
        if run_game:
            # counter = 0
            while not self.game_is_over:
                # counter += 1
                with out:
                    out.append_stdout("Computing Best Move for AI Agent \n")
                self.select_custom_move()
                time.sleep(2.5)
#                 print(f"Next Move")

    def select_custom_move(self):

        if platform.system() == "Windows":

            def curr_time_millis():
                return int(round(time.time() * 1000))

        else:

            def curr_time_millis():
                return 1000 * resource.getrusage(resource.RUSAGE_SELF).ru_utime

        move_start = curr_time_millis()

        def time_left(time_limit=1000):
            # print("Limit: "+str(time_limit) +" - "+str(curr_time_millis()-move_start))
            return time_limit - (curr_time_millis() - move_start)

        active_player = self.game.get_active_player()
#         self.output_section.append_stdout(f"Active Players Turn: {active_player}")
        
        if isinstance(active_player, RandomPlayer):
            self.output_section.append_stdout("Random Player's Turn \n")
            
        elif isinstance(active_player, CustomPlayer):
            self.output_section.append_stdout("Custom AI Player's Turn \n")
            
            start = time.time()

        ############
        # TODO: Illegal moves being allowed in the game
        all_player_moves = self.game.get_active_moves()
        player_move = active_player.move(self.game, time_left=time_left)

        if not player_move in all_player_moves:
            print(
                f"{type(active_player)} move {player_move} is not in list of legal moves {all_player_moves}"
            )

        self.game_is_over, winner = self.game.__apply_move__(player_move)

        if self.game_is_over:
            with self.output_section:
                self.output_section.append_stdout(f"Game is over, the winner is: {winner} \n")

        board_vis_state = get_viz_board_state(self.game, self.show_legal_moves)
        for r in range(self.height):
            for c in range(self.width):
                new_name, new_style = get_details(board_vis_state[r][c])
                self.gridb[r, c].description = new_name
                self.gridb[r, c].style = new_style
        self.__reset_turn()

        if isinstance(active_player, CustomPlayer):
            end = time.time()
            with self.output_section:
                self.output_section.append_stdout(f"Run time to compute move: {end - start} \n")

    def select_move(self, b):
        if platform.system() == 'Windows':
            def curr_time_millis():
                return int(round(time.time() * 1000))
        else:
            def curr_time_millis():
                return 1000 * resource.getrusage(resource.RUSAGE_SELF).ru_utime
        move_start = curr_time_millis()

        def time_left(time_limit = 1000):
            # print("Limit: "+str(time_limit) +" - "+str(curr_time_millis()-move_start))
            return time_limit - (curr_time_millis() - move_start)

        global ig
        if isinstance(ig.player1, HumanPlayer) and isinstance(ig.opponent, HumanPlayer):
            with ig.output_section:
                out.append_stdout("Human Vs. Human")
                
        

        self.__move.append((b.x, b.y))
        with self.output_section:
            self.output_section.append_stdout(f"Move {self.__click_count + 1}: {b.x}, {b.y} \n")
    
        if self.__click_count < 2:
            self.__click_count += 1
            return

#         self.output_section.append_stdout("Test1")
        if self.game_is_over:
            with self.output_section:
                self.output_section.append_stdout("The game is over! \n")
            return
        ### swap move workaround ###
        # find if current location is in the legal moves
        # legal_moves is of length 1 if move exists, and len 0 if move is illegal
        moves = self.game.get_active_moves()
        legal_moves = [(x,y,z) for x,y,z in moves if [x,y,z] == self.__move]
        if not legal_moves:
            output = f"move {self.__move} is illegal!"
            self.__reset_turn()
            with self.output_section:
                self.output_section.append_stdout(output)
            return
        else:
            # there is only one move in swap isolation game
            self.__move = legal_moves[0]
        
#         self.output_section.append_stdout("Test2")
        ### swap move workaround end ###
        self.game_is_over, winner = self.game.__apply_move__(self.__move)
        if (not self.game_is_over) and (type(self.opponent) != Player):
            opponents_legal_moves = self.game.get_active_moves()
            opponent_move = self.opponent.move(self.game, time_left=time_left)
            
            assert opponent_move in opponents_legal_moves, \
            f"Opponents move {opponent_move} is not in list of legal moves {opponents_legal_moves}"
            
            self.game_is_over, winner = self.game.__apply_move__(opponent_move)
        
#         self.output_section.append_stdout("Test3")
        if self.game_is_over:
#             print(f"Game is over, the winner is: {winner} \n")
            self.output_section.append_stdout(f"Game is over, the winner is: {winner} \n")
        
        board_vis_state = get_viz_board_state(self.game, self.show_legal_moves)
        for r in range(self.height):
            for c in range(self.width):
                new_name, new_style = get_details(board_vis_state[r][c])
                self.gridb[r,c].description = new_name
                self.gridb[r,c].style = new_style
                
        if self.game_is_over:
            # Remove callback functions from buttons
            for r in range(self.height):
                for c in range(self.width):
                    self.gridb[r,c].on_click(None)
                return
        # Reset turn and clear output state for next players turn
        self.__reset_turn()


class ReplayGame:
    """This class is used to replay games (only works in jupyter)"""

    def __init__(self, game, move_history, show_legal_moves=False):
        self.game = game
        self.width = self.game.width
        self.height = self.game.height
        self.move_history = move_history
        self.show_legal_moves = show_legal_moves
        self.board_history = []
        self.new_board = self.setup_new_board()
        self.gridb = create_board_gridbox(self.new_board, self.show_legal_moves)
        self.generate_board_state_history()
        self.visualized_state = None
        self.output_section = widgets.Output(layout={"border": "1px solid black"})

    def setup_new_board(self,):
        return Board(
            player_1=self.game.__player_1__,
            player_2=self.game.__player_2__,
            width=self.width,
            height=self.height,
        )

    def update_board_gridbox(self, move_i):
        board_vis_state, board_state = self.board_history[move_i]
        self.visualized_state = board_state
        for r in range(self.height):
            for c in range(self.width):
                new_name, new_style = get_details(board_vis_state[r][c])
                self.gridb[r, c].description = new_name
                self.gridb[r, c].style = new_style

    def equal_board_states(self, state1, state2):
        for r in range(self.height):
            for c in range(self.width):
                if state1[r][c] != state2[r][c]:
                    return False
        return True

    def generate_board_state_history(self,):
        for move_pair in self.move_history:
            for move in move_pair:
                self.new_board.__apply_move__(move[0])
                board_vis_state = get_viz_board_state(self.new_board, self.show_legal_moves)
                board_state = self.new_board.get_state()
                self.board_history.append(
                    (copy.deepcopy(board_vis_state), copy.deepcopy(board_state))
                )
        assert self.equal_board_states(
            self.game.get_state(), self.new_board.get_state()
        ), "End game state based of move history is not consistent with state of the 'game' object."

    def get_board_state(self, x):
        """You can use this state to with game.set_state() to replicate same Board instance."""
        self.output_section.clear_output()
        with self.output_section:
            display(self.visualized_state)

    def show_board(self):
        # Show slider for move selection
        input_move_i = widgets.IntText(layout=Layout(width="auto"))
        slider_move_i = widgets.IntSlider(
            description=r"\(move[i]\)",
            min=0,
            max=len(self.board_history) - 1,
            continuous_update=False,
            layout=Layout(width="auto"),
        )
        mylink = widgets.link((input_move_i, "value"), (slider_move_i, "value"))
        slider = VBox([input_move_i, interactive(self.update_board_gridbox, move_i=slider_move_i)])

        get_state_button = Button(description="get board state")
        get_state_button.on_click(self.get_board_state)

        grid = GridspecLayout(4, 6)  # , width='auto')
        # Left side
        grid[:3, :-3] = self.gridb
        grid[3, :-3] = slider

        # Right side
        grid[:-1, -3:] = self.output_section
        grid[-1, -3:] = get_state_button
        display(grid)


In [4]:
#######################################################################################
# App Bar and Toolbar Content
#######################################################################################
play_isolation_nav_bar_btn = v.ListItem(
    children=[
        v.ListItemIcon(children=[v.Icon(children=['mdi-play'])]),
        v.ListItemTitle(children=['Play Isolation'])
    ],
)

navDrawer = v.NavigationDrawer(
    _metadata={'mount_id': 'content-nav'},
    permanent=True,
    value="Play Isolation",
    height="90vh",
    children=[
        v.ListItemGroup(
            active_class="highlighted",
#             style_="length:50%",
            children = [
                play_isolation_nav_bar_btn,
                v.ListItem(
                    children=[
                        v.ListItemIcon(children=[v.Icon(children=['mdi-school'])]),
                        v.ListItemTitle(children=['Learn More'])
                    ],
                ),
                v.ListItem(
                    children=[
                        v.ListItemIcon(children=[v.Icon(children=['mdi-google-analytics'])]),
                        v.ListItemTitle(children=['Simulation Dashboard'])
                    ],
                ),
            ]
        ),
    ]
)

toolbar = v.Toolbar(
    dense=True,
    dark=True,
    color='primary',
    children=[
         v.ToolbarTitle(children=['AI Isolation Dashboard']),
         v.Spacer(),
         ])

#######################################################################################
# Main Content - Sidebar - Player Selection & AI Agent Options
#######################################################################################
sidebar_elem_width = "50%"

sidebar_text = """

Welcome to the AI Isolation Agent Dashboard! Learn how to play the game using the 'Learn More' tab and
try to beat the AI Agent's available. Use the Player select options below to choose your opponent

Test

Test

"""

player_map = {
    "Human Player": Player(), #HumanPlayer(),
    "Random Player": RandomPlayer(),
    "Minimax": CustomPlayer(eval_fn = OpenMoveEvalFn(), search_depth = 1),
#     "Alphabeta": CustomAIPlayer() ### TODO: Need to add option to switch search method over game tree
}

ai_agent_heuristic_map = {
    "Open Move Evaluation Function": OpenMoveEvalFn(), 
    "Defensive": DefensiveEvalFn(), 
    "Offensive": OffensiveEvalFn(), 
    "Defense To Offense": DefenseToOffenseEvalFn(), 
    "Offense To Defense": OffenseToDefenseEvalFn(),
}

break_html_elem = v.Html(tag="br")
markdown_content = v.Html(tag="p", children = [sidebar_text], style_="width: 50%")

#######################################################################################
# Player & Opponent Selections
#######################################################################################

class GameOptionsSelector(v.VuetifyTemplate):
    # Vue script code for buttons and player selections
    template_file = 'game-options-selector.vue'
    
    # Values to be assigned to Vue components
    player_options = ['Human Player', 'Random Player', 'Minimax', 'Alphabeta']
    ai_agents = ['Minimax', 'Alphabeta']
    
    heuristic_options = [
        "Open Move Evaluation Function", 
        "Defensive", 
        "Offensive", 
        "Defense To Offense", 
        "Offense To Defense"
    ]
    non_ai_options = ['Human Player', 'Random Player']
    
    # Create traitlets to be used in the start-button.vue script
    players = traitlets.List(
        traitlets.Unicode(), 
        default_value = player_options
    ).tag(sync=True)
    
    selected = traitlets.Unicode(default_value=None, allow_none=True).tag(sync=True)
    selected2 = traitlets.Unicode(default_value=None, allow_none=True).tag(sync=True)
    
    ai_agent_vals = traitlets.List(traitlets.Unicode(), default_value = ai_agents).tag(sync=True)
    non_ai_agent_players = traitlets.List(traitlets.Unicode(), default_value = non_ai_options).tag(sync=True)
    ai_strategies = traitlets.List(traitlets.Unicode(), default_value=heuristic_options).tag(sync=True)
    
    non_ai_players = traitlets.List(traitlets.Unicode(), default_value=non_ai_options).tag(sync=True)
    ai_agent_strategy = traitlets.Unicode(default_value=None, allow_none=True).tag(sync=True)
    
    isDisabled = traitlets.Bool(default_value=True).tag(sync=True)
    button = traitlets.Bool(default_value=False).tag(sync=True)
    
    agent_timeout = traitlets.Integer(90).tag(sync=True)
    
    
    def vue_on_player_select(self, data = None):
        global ig
        global out
        
        player1 = player_map.get(self.selected, "")
        player2 = player_map.get(self.selected2, "")
        
        # If ai agent playing strategy is selected, use this as our move scoring function
        if self.ai_agent_strategy:
            player2.eval_fn = ai_agent_heuristic_map.get(self.ai_agent_strategy, "")
        else:
            player2.eval_fn = ai_agent_heuristic_map.get("Open Move Evaluation Function")
            
        if self.selected2 in ['Minimax', 'Alphabeta']:
            player2.output = out
        

        ig = PlayInteractiveGame(player1, player2, show_legal_moves=True, output_section = out)
        ig.callback_func = ig.select_callback_func(player1, player2)
        ig.game = Board(player1, player2)
        

        
#         with ig.output_section:
#             ig.output_section.append_stdout(f"Game Details: P1 --> {ig.game.__player_1__}, P2 --> {ig.game.__player_2__}")
#             ig.output_section.append_stdout(f"Player 1: {type(player1)}, Player 2: {type(player2)}")
#             ig.output_section.append_stdout(f"Callback Function: {ig.callback_func}")

        global game_header
        game_header.children = [f"Player1: {self.selected} Vs. Player 2: {self.selected2}"]

        # Update children for Isolation grid
        global grid_container
        
        grid_container.children = [
            game_header,
            break_html_elem,
            ig.gridb, 
#             ig.output_section
        ]

        app.widgets.update()
        
#         ai_vs_cpu = (
#             (isinstance(player1, RandomPlayer) and isinstance(player2, CustomAIPlayer)) or 
#             (isinstance(player1, CustomAIPlayer) and isinstance(player2, RandomPlayer))
#         )

        if isinstance(player1, RandomPlayer) and isinstance(player2, CustomPlayer):
            with out:
                out.append_stdout("On Start Button: Starting CPU/AI Game..... \n")
                out.append_stdout(f"AI Agent Strategy: {player2.eval_fn.__name__}")
            return ig.run_cpu_ai_game(run_game = True)

#         with out:
#             out.clear_output()

In [5]:
#######################################################################################
# Global Variables
#######################################################################################

app = v.App(v_model=None)

null_game = Board(None, None)

out = widgets.Output(
    layout={
        'border': 'none',
#         'border': '1px solid black', 
        'overflow_y': 'auto',
        'height': '480px',
        'width': '480px'
    }
)
# ig = PlayInteractiveGame(HumanPlayer(), RandomPlayer(), show_legal_moves=True, output_section = out)
ig = PlayInteractiveGame(None, None, show_legal_moves=True, output_section = out)

#######################################################################################
# Instantiate Veutify Objects for use in the App
#######################################################################################


# Game Options & Start Sidebar
game_options_selector = GameOptionsSelector()
# start_button.observe(start_game_on_click, names="selected")

#######################################################################################
# Isolation Grid Functions
#######################################################################################

game_header = v.Html(tag = "h2", children = ["Player 1: Vs. Player 2: "])

grid_container = v.Container(
    id = "game-container",
    tag="gamegrid",
    fluid = True, 
    style_ = "height:90%", 
    children = [
        game_header,
        break_html_elem,
        # Output widget
        ig.gridb, 
#         ig.output_section
    ]
)

output_container = v.Container(
    fluid = True,
    children = [
        v.Html(tag="h2", children = ["Game Logs"]),
        break_html_elem,
        ig.output_section,
    ]
)

#######################################################################################
# Output Content
#######################################################################################


def get_output_content(output_widget):
    """ Can access each new addition to the output widget as the 'text' key for each tuple
    """
    outerlist = []
    items = []
    for elem in output_widget.outputs:
        items.append(elem.get("text", "") + "\n")
    
    for item in items:
        outerlist.append([item])

    return outerlist

output_card = v.Card(class_='ma-2 pa-0',
                     hover=True,
                     outlined=True,
                     width=500,
                     height=500,
            #        color='deep-orange lighten-2',
                     target='none',
                     children=[
                         v.CardTitle(class_='headline gray lighten-8',
                                     primary_title=True,
                                     children=["Game Logs"]),
                         v.CardText(children = [out])
                     ]
)

output_card_container = v.Container(
    fluid = True, 
    children = [
        break_html_elem, 
        output_card
    ], 
    style_="align-items:center"
)

#######################################################################################
# Layout Content Containers
#######################################################################################

content_container = v.Container(
    _metadata={'mount_id': 'content-main'},
    style_="margin-left:1vh",
    fluid=True,
    children = [
        v.Row(
            no_gutters=True,
            align_items = "baseline",
            children = [
                v.Col(
                    cols=4,
                    children= [
                        markdown_content,
                        break_html_elem,
                        game_options_selector,
                ]), 
                v.Col(cols=4, children = [grid_container]),
                v.Col(cols = 4, children = [output_card_container]),
            ]),
        v.Row(
            no_gutters=True,
            align_items = "baseline",
            children = [
                v.Col(
                    cols = 4,
                    children = [
                    ]
                )
            ]
        )
    ]
)

main_page_content_container = v.Container(
    fluid = True,
    children = [
        v.Row(
            align = "baseline",
            align_content="start",
            justify="start",
            class_="flex-nowrap shrink",
            children = [
#                 v.Col(
#                     cols=1,
#                     children = [
#                         navDrawer,
#                     ]
#                 ),
                v.Col(cols=12, children = [content_container])
            ])
    ]
)

#######################################################################################
# Learn More Tab Content
#######################################################################################

isolation_agent_text = """
Isolation is a deterministic, two-player game in which the players alternate turns moving a single piece from one cell to another, on a board. Whenever either player occupies a cell, that cell becomes 
blocked for the remainder of the game. The first player with no remaining legal moves loses, and the opponent is declared the winner.

This project is an attempt at building a game playing 'adversarial search' agent for Isolation combining different strategies. Concepts covered in this project include:

- Building a game tree 
- Minimax search
- Alphabeta Pruning (optimization of minimax)
- Iterative deepening (time-limited)
- Evaluation functions
"""


game_instructions = """
The rules of 3 Snails Isolation are a variation of the original Isolation. In the original form of the game there are two players, each with their own game piece, and a 7-by-7 grid of squares. 
At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, 
the players alternate turns moving their piece like a queen in chess (any number of open squares vertically, horizontally, or diagonally). When the piece is moved, the square that was previously 
occupied is blocked, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this 3 Snails variant, each player controls 3 pieces ("snails"), each of which can only move to 1 surrounding square, either to the left/right or directly above/below it. Once a piece moves away 
from a square, no other piece can occupy the square again, as in the original Isolation game. For clarity, examine the scenario below:

In the image below, both Q1 and Q2 place their snails on the board. Note, that when choosing your move, you must ensure your moves are selected in proper order
"""

game_instructions_2 = """
In the image below, Q1 is in the process of choosing its next set of moves. It has chosen 2 moves already, which are printed in the output box, and still needs to pick where "13" will move. Note that 
the possible moves for any given piece is either to the left/down or directly above/below it.
"""

game_instructions_3 = """
Q1 makes its move and Q2 has already chosen the next set of moves. In the image below, Q2 tried to move two of its snails to the same location. Such moves are illegal, as indicated by the message in 
the output box.
"""

game_instructions_4 = """
You can try playing the game against the Random Player or yourself using the interactive tool located in the 'Play Isolation Tab'.
"""

picture_height = 600
picture_width = 500
n_cols = 10


about_the_game_container = v.Container(
    fluid = True,
    children = [
    v.Row(
        align_content="center",
        justify="center",
        children = [
            v.Col(
            cols = n_cols, 
            children = [
                v.Html(
                    tag="h1", 
                    children = ["About The Game"]),
                    v.Divider(),
                ]), 
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    v.Html(tag = "p", class_ = "text-block", style_ = "white-space: pre; font-size: 18px", children = [game_instructions]),
                    v.Img(src="./img/sn_1.png", max_height = picture_height, max_width = picture_width, contain = True),
            ]),
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    v.Html(tag = "p", class_ = "text-block", style_ = "white-space: pre; font-size: 18px", children = [game_instructions_2]),
                    v.Img(src="./img/sn_2.png", max_height = picture_height, max_width = picture_width),
            ])
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    v.Html(tag = "p", class_ = "text-block", style_ = "white-space: pre; font-size: 18px", children = [game_instructions_3]),
                    v.Img(src="./img/sn_3.png", max_height = picture_height, max_width = picture_width)
            ])
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    v.Html(tag = "p", class_ = "text-block", style_ = "white-space: pre; font-size: 18px", children = [game_instructions_4]),
            ])
        ]),
])

game_playing_text = """
As the game board in Isolation is fully viewable by both players at all times, the game is fully observable, adversarial, and deterministic. This allows for the 
development of an AI-based game playing agent using suitable algorithms and heuristics, such as Minimax and Alphabeta Pruning
"""

minimax_text = """
Minimax is a decision-making algorithm, typically used in a turn-based, two player games. The goal of the algorithm is to find the optimal next move.

- Idea: Choose move to position with highest minimax value = best achievable payoff against best play
- Perfect (optimal) play for deterministic, perfect-information games
"""

minimax_algorithm_summary = """
In the algorithm, one player is called the maximizer, and the other  player is a minimizer. If we assign an evaluation score to the game board, one player tries to 
choose a game state with the maximum score, while the other chooses a state with the minimum score. In other words, the maximizer works to get the highest score, while 
the minimizer tries get the lowest score** by trying to counter moves

It is based on the zero sum game concept. In a zero-sum game, the total utility score is divided among the players. 
An increase in one player's score results into the decrease in another  player's score. So, the total score is always zero. For one  player to win, the other one has 
to lose. Examples of such games are chess, poker, checkers, tic-tac-toe

- We traverse the game tree depth-first until we reach the terminal nodes and assign their parent node a value that is best for the player whose turn it is to move. 
For example, the game tree of a game of tic-tac-toe looks like:
"""



game_playing_content = v.Container(
    fluid = True, 
    children = [
        v.Row(
            align_content="center",
            justify="center",
            children = [
                v.Col(
                    cols = n_cols, 
                    children = [
                        v.Html(tag="h1", children = ["Game Playing"]),
                        v.Divider(),
                ]),                
        ]),
        v.Row(
            align_content="center",
            justify="center",
            children = [
                v.Col(
                    cols = n_cols, 
                    children = [
                        v.Html(
                            tag = "p", 
                            class_ = "text-block", 
                            style_ = "white-space: pre; font-size: 18px", 
                            children = [ 
                                game_playing_text

                            ]),
                ]),                
        ]),        
        v.Row(
            align_content="center",
            justify="center",            
            children = [
                v.Col(
                    cols = n_cols, 
                    children = [
                        v.Html(tag="h3", children = ["Minimax Algorithm"]),
                        v.Divider(),
                        v.Html(
                            tag = "p", 
                            class_ = "text-block", 
                            style_ = "white-space: pre; font-size: 18px", 
                            children = [ 
                                minimax_text,
                                minimax_algorithm_summary
                            ]),
                        break_html_elem,
                        v.Img(class_ = "mx-auto", src="./img/tictactoe-gametree.png", max_height = picture_height * 2, max_width = picture_width * 2, contain = True),
                ]),
        ]),
        
])

exp_panel1 = v.ExpansionPanel(
    children=[
        v.ExpansionPanelHeader(style_= "font-size: 24px", children=['3 Snails Isolation Game Rules']),
        v.ExpansionPanelContent(children=[about_the_game_container])])

exp_panel2 = v.ExpansionPanel(children=[
    v.ExpansionPanelHeader(style_= "font-size: 24px", children=['The Minimax & Alphabeta Algorithms']),
    v.ExpansionPanelContent(children=[game_playing_content])])

vep = v.ExpansionPanels(children=[exp_panel1, exp_panel2])
vl = v.Layout(class_='pa-4', children=[vep])



learn_more_content = v.Container(
    fluid = True, 
    children = [
        v.Row(
            align_content="center",
            justify="center",
            children = [
                v.Col(
                    cols = n_cols, 
                    children = [
                        v.Html(tag="h1", children = ["Isolation - Game Playing Agent"]),
                        v.Divider(),
                ]),                
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    v.Html(
                        tag = "p", 
                        class_ = "text-block", 
                        style_ = "white-space: pre; font-size: 18px",
                        children = [isolation_agent_text]),
            ]),
        ]),
        v.Row(
            align_content="center",
            justify="center",            
            children = [
            v.Col(
                cols = n_cols, 
                children = [
                    vl
            ]),
        ]),
        
])




#######################################################################################
# Simulation Tab Content
#######################################################################################



#######################################################################################
# Tab Content
#######################################################################################

play_isolation_tab = v.Tab(children = ["Play Isolation"])
play_isolation_content = v.TabItem(children = [content_container])

learn_more_tab = v.Tab(children = ["Learn More"])
learn_more_content = v.TabItem(children = [learn_more_content])

simulation_tab = v.Tab(children = ["Simulation"])
simulation_content = v.TabItem(children = [])


tab_list = [play_isolation_tab, learn_more_tab, simulation_tab]
# content_list = [v.TabItem(children=['Tab Content']) for i in range(3)]
content_list = [play_isolation_content, learn_more_content, simulation_content]
tabs = v.Tabs(v_model=0, children=tab_list + content_list, centered=True, grow = True)

#######################################################################################
# Assemble Content
#######################################################################################


main_container = v.Container(
    _metadata={'mount_id': 'content'},
    fluid=True,
    style_='padding : 1px 1px 1px 1px',
    class_="py-0 px-0",
    children = [toolbar, tabs] #, main_page_content_container]
)

#######################################################################################
# Finalize Layout of the app
#######################################################################################

app.children = [main_container]
app

App(children=[Container(children=[Toolbar(children=[ToolbarTitle(children=['AI Isolation Dashboard']), Spacer(…

NotImplementedError: Random (P1) vs Human (P2) not implemented

In [ ]:



# def curr_time_millis():
#     return 1000 * resource.getrusage(resource.RUSAGE_SELF).ru_utime

# move_start = curr_time_millis()
# move_start

In [ ]:
# def time_left(time_limit=1000):
#     # print("Limit: "+str(time_limit) +" - "+str(curr_time_millis()-move_start))
#     return time_limit - (curr_time_millis() - move_start)

# time_left()

In [ ]:
out = widgets.Output(
    layout={
        'font-family': 'Roboto',
        'justify-content': 'left',
        'border': 'None', 
        'overflow_y': 'auto',
        'height': '400px',
        'width': '480px',
    }
)
with out:
    out.append_stdout("Test")
    out.append_stdout("Test2")
out

In [ ]:
# dir(out)

In [ ]:
# def get_output_content(output_widget):
#     """ Can access each new addition to the output widget as the 'text' key for each tuple
#     """
#     outerlist = []
#     items = []
#     for elem in output_widget.outputs:
#         items.append(elem.get("text", "") + "\n")
    
#     for item in items:
#         outerlist.append([item])

#     return outerlist

# output_card = v.Card(class_='ma-2 pa-0',
#                      hover=True,
#                      outlined=True,
#                      width=400,
#                      height=400,
#             #        color='deep-orange lighten-2',
#                      target='none',
#                      children=[
#                          v.CardTitle(class_='headline gray lighten-8',
#                                      primary_title=True,
#                                      children=["Game Logs"]),
#                          v.CardText(children = [out])
#                      ]
# )